In [ ]:
!pip install -q diffusers transformers accelerate safetensors torch torchvision


In [ ]:
import torch
import random
import os
from diffusers import StableDiffusionXLPipeline


In [ ]:
import torch
import random
from diffusers import StableDiffusionXLPipeline
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
!pip install -q xformers
# Enable CPU offload to save GPU memory


In [ ]:
# Load SDXL model
model_id = "stabilityai/stable-diffusion-xl-base-1.0"

pipe = StableDiffusionXLPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
)

# Enable CPU offload to save GPU memory
pipe.enable_model_cpu_offload()

# xformers disabled due to Colab compatibility
print("SDXL loaded successfully (xformers disabled)")

In [ ]:
# Block 1: Mount Google Drive and create project structure

from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define project root directory on Google Drive
PROJECT_ROOT = "/content/drive/MyDrive/hazard_detection_genai"

# Define project subdirectories
DIRS = {
    "raw_images": os.path.join(PROJECT_ROOT, "data/images/raw"),
    "train_images": os.path.join(PROJECT_ROOT, "data/images/train"),
    "val_images": os.path.join(PROJECT_ROOT, "data/images/val"),
    "train_labels": os.path.join(PROJECT_ROOT, "data/labels/train"),
    "val_labels": os.path.join(PROJECT_ROOT, "data/labels/val"),
    "notebooks": os.path.join(PROJECT_ROOT, "notebooks"),
    "docs": os.path.join(PROJECT_ROOT, "docs")
}

# Create all directories
for path in DIRS.values():
    os.makedirs(path, exist_ok=True)

print("✅ Project structure created successfully:")
for name, path in DIRS.items():
    print(f"{name}: {path}")


In [ ]:
IMAGE_SIZE = 1024
STEPS = 30
CFG_SCALE = 6.5

NEGATIVE_PROMPT = (
    "cartoon, illustration, anime, 3d render, low quality, blurry, "
    "distorted objects, unrealistic lighting, oversaturated, "
    "people, hands, faces, text, watermark"
)


In [ ]:
OBJECT_PROMPTS = {
    "battery": [
        "a small AA battery lying on the wooden floor near a toy shelf",
        "a small battery on a carpet in a children's room",
        "a battery partially under a chair in a child's bedroom"
    ],
    "coin": [
        "a small coin on the floor next to scattered children's toys",
        "a coin near the bed in a children's room"
    ],
    "small_toy_part": [
        "a small plastic toy part on the floor, detached from a children's toy",
        "a tiny toy component near a toy box"
    ],
    "scissors": [
        "a pair of scissors partially visible on a child's desk",
        "scissors placed on a low table in a children's room"
    ],
    "knife": [
        "a small kitchen knife placed on a low shelf within reach of a child",
        "a small knife lying on a desk in a children's bedroom"
    ],
    "cleaning_bottle": [
        "a cleaning product bottle standing on the floor near a wardrobe",
        "a household cleaning bottle under a desk in a children's room"
    ],
    "detergent_container": [
        "a detergent container placed under a small table in a children's room",
        "a laundry detergent container near the wall"
    ],
    "power_cable": [
        "a loose electrical power cable lying on the floor near the wall",
        "an electrical cable partially coiled on the floor"
    ],
    "electrical_socket": [
        "a low wall electrical socket visible near the floor in a children's room",
        "an electrical wall socket close to the ground"
    ]
}


In [ ]:
def generate_image(image_id: int):
    # Select 1–3 objects randomly
    num_objects = random.choice([1, 2, 3])
    selected_objects = random.sample(list(OBJECT_PROMPTS.keys()), num_objects)

    object_descriptions = []
    for obj in selected_objects:
        object_descriptions.append(random.choice(OBJECT_PROMPTS[obj]))

    objects_text = ", ".join(object_descriptions)

    prompt = (
        "Photorealistic children's bedroom, modern apartment, "
        "natural daylight, realistic shadows, DSLR photo, "
        "high detail, realistic textures, "
        f"{objects_text}, "
        "no people"
    )

    image = pipe(
        prompt=prompt,
        negative_prompt=NEGATIVE_PROMPT,
        height=IMAGE_SIZE,
        width=IMAGE_SIZE,
        num_inference_steps=STEPS,
        guidance_scale=CFG_SCALE
    ).images[0]

    filename = f"img_{image_id:04d}.png"
    image.save(os.path.join(DIRS["raw_images"], filename))
def generate_image(image_id: int):
    # Select 1–3 objects randomly
    num_objects = random.choice([1, 2, 3])
    selected_objects = random.sample(list(OBJECT_PROMPTS.keys()), num_objects)

    object_descriptions = []
    for obj in selected_objects:
        object_descriptions.append(random.choice(OBJECT_PROMPTS[obj]))

    objects_text = ", ".join(object_descriptions)

    prompt = (
        "Photorealistic children's bedroom, modern apartment, "
        "natural daylight, realistic shadows, DSLR photo, "
        "high detail, realistic textures, "
        f"{objects_text}, "
        "no people"
    )

    image = pipe(
        prompt=prompt,
        negative_prompt=NEGATIVE_PROMPT,
        height=IMAGE_SIZE,
        width=IMAGE_SIZE,
        num_inference_steps=STEPS,
        guidance_scale=CFG_SCALE
    ).images[0]

    filename = f"img_{image_id:04d}.png"
    image.save(os.path.join(DIRS["raw_images"], filename))


In [ ]:
NUM_IMAGES = 30  #  30–50 для проверки качества

for i in range(NUM_IMAGES):
    generate_image(i)
    print(f"Generated image {i+1}/{NUM_IMAGES}")


In [ ]:
import os


existing_files = os.listdir(DIRS["raw_images"])
existing_count = len(existing_files)

print(f"Found {existing_count} existing images in raw_images folder.")

# Number of new images to generate
NEW_IMAGES = 300

for i in range(existing_count, existing_count + NEW_IMAGES):
    generate_image(i)
    if (i - existing_count + 1) % 10 == 0:
        print(f"Generated {i - existing_count + 1}/{NEW_IMAGES} new images")

print(" Phase 1 image generation complete")
